# Travel Agent API - Quick Test Notebook
Use this notebook to smoke-test the FastAPI backend endpoints.
- Make sure the server is running (see next cell for a reminder).
- Requires: `requests` package.

## Start the API server (outside this notebook)
In a terminal (Windows cmd), activate the conda env, set the API key, and run the backend:

```cmd
conda activate gen-ai
set TRAVEL_AGENT_API_KEY=YOUR_SECRET_KEY
python -m backend.main
```

Default base URL: http://localhost:8000

Notes:
- Authenticated endpoint: POST /api/chat requires the API key via `Authorization: Bearer <key>` (used by this notebook) or `X-API-Key: <key>`.
- GET /health is open (no auth).

In [1]:
import os, json, time
import requests
from dotenv import load_dotenv
load_dotenv()
BASE_URL = os.getenv('TRAVEL_AGENT_API_BASE', 'http://localhost:8000')
API_KEY = os.getenv('TRAVEL_AGENT_API_KEY')  # required for authenticated endpoints

s = requests.Session()
if API_KEY:
    # Use Bearer scheme; backend also accepts X-API-Key
    s.headers.update({'Authorization': f'Bearer {API_KEY}'})

print('Using base URL:', BASE_URL)
print('Auth header set:', 'YES' if API_KEY else 'NO (set TRAVEL_AGENT_API_KEY)')
print('Session created; cookies will persist automatically.')

Using base URL: http://localhost:8000
Auth header set: YES
Session created; cookies will persist automatically.


In [8]:
# Health check (no auth required)
resp = s.get(f'{BASE_URL}/health', timeout=10)
print(resp.status_code, resp.text)
resp.raise_for_status()
resp.json()

200 {"status":"ok"}


{'status': 'ok'}

In [3]:
# Send a chat message (establish cookie-based session)
if not os.getenv('TRAVEL_AGENT_API_KEY'):
    raise RuntimeError('TRAVEL_AGENT_API_KEY environment variable is required for /api/chat')

payload = {
    'message': 'Plan a 10-day trip to Japan in November with luxury hotels',
    'reset': True,
}
resp = s.post(f'{BASE_URL}/api/chat', json=payload, timeout=120)
print(resp.status_code)
print(resp.text[:1000])
resp.raise_for_status()
data = resp.json()
data

200
{"response":"Here are luxury hotel options in Kyoto and Osaka for your 10-day November trip:\n\n**Kyoto**\n- Park Hyatt Kyoto (5-star): $1,707/night, prime Gion location, spa & fine dining\n- THE JUNEI HOTEL Kyoto (5-star): $516/night, boutique, cedar soaking tubs\n- The Celestine Hotel Gion (4.5-star): $429/night, elegant, bathhouse, shuttle\n\n**Osaka**\n- Swissôtel Nankai Osaka (5-star): $288/night, spa, gym, 6 restaurants, direct train access\n- Centara Grand Hotel Osaka (5-star): $284/night, spa, gourmet dining, fitness center\n- Royal Park Hotel Iconic Osaka-Midosuji (5-star): $213/night, French restaurant, gym\n\nWould you like to split your stay between Kyoto and Osaka, or stay in one city only? Do you need luxury flight options as well?\n\n### Flights\nWould you like me to arrange flights for this trip as well? If so, please specify your departure city/airport.\n\nFor luxury accommodations, the best 5-star choices for your November 10-day trip are:\n- Kyoto: Park Hyatt Kyo

{'response': 'Here are luxury hotel options in Kyoto and Osaka for your 10-day November trip:\n\n**Kyoto**\n- Park Hyatt Kyoto (5-star): $1,707/night, prime Gion location, spa & fine dining\n- THE JUNEI HOTEL Kyoto (5-star): $516/night, boutique, cedar soaking tubs\n- The Celestine Hotel Gion (4.5-star): $429/night, elegant, bathhouse, shuttle\n\n**Osaka**\n- Swissôtel Nankai Osaka (5-star): $288/night, spa, gym, 6 restaurants, direct train access\n- Centara Grand Hotel Osaka (5-star): $284/night, spa, gourmet dining, fitness center\n- Royal Park Hotel Iconic Osaka-Midosuji (5-star): $213/night, French restaurant, gym\n\nWould you like to split your stay between Kyoto and Osaka, or stay in one city only? Do you need luxury flight options as well?\n\n### Flights\nWould you like me to arrange flights for this trip as well? If so, please specify your departure city/airport.\n\nFor luxury accommodations, the best 5-star choices for your November 10-day trip are:\n- Kyoto: Park Hyatt Kyoto,

In [ ]:
# Follow-up message in same session (cookies persist)
if not os.getenv('TRAVEL_AGENT_API_KEY'):
    raise RuntimeError('TRAVEL_AGENT_API_KEY environment variable is required for /api/chat')

payload = {
    'message': 'November 18-21 departure city AUS for two adults and I don\'t care about the area',
    'reset': False,
}
resp = s.post(f'{BASE_URL}/api/chat', json=payload, timeout=120)
print(resp.status_code)
print(resp.text[:1000])
resp.raise_for_status()
data = resp.json()
data

200
{"response":"Confirmed: Nov 18–21, AUS to Barcelona, 2 adults, no area preference, luxury hotel.\n\n**Flights:**\n- Business class, $1,126–$1,253 per person, typically 3 layovers (routes via Miami, Philadelphia, London).\n\n**Top Luxury Hotels:**\n1. NH Collection Barcelona Gran Hotel Calderón (5-star), $823 total\n2. Suites Avenue (designer serviced apartment), $740 total\n3. Hotel Duquesa de Cardona (upscale, port view), $756 total\n\nWould you like to see detailed flight schedules/layovers, or proceed with a hotel booking? Please specify your preferred hotel if ready to book.\n\n### Flights\nHere are your options for a 4-day luxury Barcelona trip, departing AUS (Austin) Nov 18 and returning Nov 21 for 2 adults:\n\n**Business Class Flights**\n- American Airlines/British Airways combo, round-trip: $1,126–$1,253 per person\n- Routing: AUS–MIA–BCN, return via London/Philadelphia/Miami\n- Multiple layovers each way, duration averages 29–32 hours each way\n\n**Top Luxury Hotels**\n1. 

{'response': 'Confirmed: Nov 18–21, AUS to Barcelona, 2 adults, no area preference, luxury hotel.\n\n**Flights:**\n- Business class, $1,126–$1,253 per person, typically 3 layovers (routes via Miami, Philadelphia, London).\n\n**Top Luxury Hotels:**\n1. NH Collection Barcelona Gran Hotel Calderón (5-star), $823 total\n2. Suites Avenue (designer serviced apartment), $740 total\n3. Hotel Duquesa de Cardona (upscale, port view), $756 total\n\nWould you like to see detailed flight schedules/layovers, or proceed with a hotel booking? Please specify your preferred hotel if ready to book.\n\n### Flights\nHere are your options for a 4-day luxury Barcelona trip, departing AUS (Austin) Nov 18 and returning Nov 21 for 2 adults:\n\n**Business Class Flights**\n- American Airlines/British Airways combo, round-trip: $1,126–$1,253 per person\n- Routing: AUS–MIA–BCN, return via London/Philadelphia/Miami\n- Multiple layovers each way, duration averages 29–32 hours each way\n\n**Top Luxury Hotels**\n1. NH 

In [ ]:
# Another follow-up in same session (cookies persist)
if not os.getenv('TRAVEL_AGENT_API_KEY'):
    raise RuntimeError('TRAVEL_AGENT_API_KEY environment variable is required for /api/chat')

payload = {
    'message': 'yes please provide me woith the first option for the flights and first options for the hotel and give me a table format with the totals',
    'reset': False,
}
resp = s.post(f'{BASE_URL}/api/chat', json=payload, timeout=120)
print(resp.status_code)
print(resp.text[:1000])
resp.raise_for_status()
data = resp.json()
data

200
{"response":"Here’s your Barcelona trip package summary:\n\n| Item    | Description                                                                            | Guests | Unit Price (USD) | Total Price (USD) |\n|---------|----------------------------------------------------------------------------------------|--------|------------------|-------------------|\n| Flight  | AUS–BCN round-trip, business class (American/BA combo, 3 layovers)                     |   2    | $1,126           | $2,252            |\n| Hotel   | NH Collection Barcelona Gran Hotel Calderón, Nov 18–21 (5-star, 3 nights, city center)  |   2    | $274/night       | $823              |\n| **Total** |                                                                                    |        |                  | **$3,075**        |\n\n- [NH Collection Barcelona Gran Hotel Calderón booking link](https://www.nh-hotels.com/en/hotel/nh-collection-barcelona-gran-hotel-calderon?utm_campaign=local-gmb&utm_medium=organic_sea

{'response': "Here’s your Barcelona trip package summary:\n\n| Item    | Description                                                                            | Guests | Unit Price (USD) | Total Price (USD) |\n|---------|----------------------------------------------------------------------------------------|--------|------------------|-------------------|\n| Flight  | AUS–BCN round-trip, business class (American/BA combo, 3 layovers)                     |   2    | $1,126           | $2,252            |\n| Hotel   | NH Collection Barcelona Gran Hotel Calderón, Nov 18–21 (5-star, 3 nights, city center)  |   2    | $274/night       | $823              |\n| **Total** |                                                                                    |        |                  | **$3,075**        |\n\n- [NH Collection Barcelona Gran Hotel Calderón booking link](https://www.nh-hotels.com/en/hotel/nh-collection-barcelona-gran-hotel-calderon?utm_campaign=local-gmb&utm_medium=organic_search

In [ ]:
if not os.getenv('TRAVEL_AGENT_API_KEY'):
    raise RuntimeError('TRAVEL_AGENT_API_KEY environment variable is required for /api/chat')

payload = {
    'message': 'departure AUS and destnation rome',
    'reset': False,
}
resp = s.post(f'{BASE_URL}/api/chat', json=payload, timeout=120)
print(resp.status_code)
print(resp.text[:1000])
resp.raise_for_status()
data = resp.json()
data

200
{"response":"Confirmed:\n- Departure: Austin, TX (AUS)\n- Destination: Rome (FCO)\n- Dates: November 18–28\n- 2 adults, budget accommodations (private room), one museum per day\n\n[Delegate: FlightSpecialist]\nPlease return 2–3 roundtrip flight options from AUS to FCO for 2 adults, departing Nov 18 and returning Nov 28, prioritizing budget and reasonable travel times.\n\n[Delegate: HotelSpecialist]\nPlease provide 3 centrally located Rome hotel/hostel options with private rooms for 2 adults, under $100/night, for 10 nights, with total pricing.\n\nOnce I have the flight and hotel options, I’ll present sample museum itineraries with entry fees for your complete budgeting."}


{'response': 'Confirmed:\n- Departure: Austin, TX (AUS)\n- Destination: Rome (FCO)\n- Dates: November 18–28\n- 2 adults, budget accommodations (private room), one museum per day\n\n[Delegate: FlightSpecialist]\nPlease return 2–3 roundtrip flight options from AUS to FCO for 2 adults, departing Nov 18 and returning Nov 28, prioritizing budget and reasonable travel times.\n\n[Delegate: HotelSpecialist]\nPlease provide 3 centrally located Rome hotel/hostel options with private rooms for 2 adults, under $100/night, for 10 nights, with total pricing.\n\nOnce I have the flight and hotel options, I’ll present sample museum itineraries with entry fees for your complete budgeting.'}

In [4]:
# Deep-debug SerpApi Google Flights call (single cell)
import os, sys, time, json, socket, ssl, platform, urllib.parse, logging
import requests, certifi

# ---------- Configuration ----------
TIMEOUT = 30  # seconds
RAW_URL = " https://serpapi.com/search?engine=google_hotels&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&currency=USD&hl=en&gl=us"
ENV_API_KEY = os.getenv("SERPAPI_API_KEY")  # if set, will be used in attempt #2
BASE_URL = "https://serpapi.com/search"

# Optional: enable connection-level logs from urllib3 (requests backend)
logging.basicConfig(level=logging.INFO)
logging.getLogger("urllib3").setLevel(logging.DEBUG)

def redact_url(url: str) -> str:
    parts = urllib.parse.urlsplit(url)
    qs = urllib.parse.parse_qsl(parts.query, keep_blank_values=True)
    redacted_qs = []
    for k, v in qs:
        if k.lower() == "api_key":
            redacted_qs.append((k, "***REDACTED***"))
        else:
            redacted_qs.append((k, v))
    return urllib.parse.urlunsplit((parts.scheme, parts.netloc, parts.path, urllib.parse.urlencode(redacted_qs), parts.fragment))

def parse_query(url: str) -> dict:
    return dict(urllib.parse.parse_qsl(urllib.parse.urlsplit(url).query, keep_blank_values=True))

def print_env_and_connectivity():
    print("=== Environment ===")
    print(f"OS: {platform.system()} {platform.release()} ({platform.version()})")
    print(f"Python: {sys.version.split()[0]}")
    print(f"requests: {requests.__version__}")
    print(f"certifi: {certifi.where()}")
    print(f"SERPAPI_API_KEY in env: {'YES' if ENV_API_KEY else 'NO'}")
    print("\n=== DNS/TLS connectivity ===")
    try:
        ips = sorted({ai[4][0] for ai in socket.getaddrinfo("serpapi.com", 443)})
        print(f"serpapi.com resolves to: {', '.join(ips)}")
    except Exception as e:
        print(f"DNS resolution failed: {e}")
    try:
        ctx = ssl.create_default_context()
        with socket.create_connection(("serpapi.com", 443), timeout=5) as sock:
            with ctx.wrap_socket(sock, server_hostname="serpapi.com") as ssock:
                proto = ssock.version()
                cipher = ssock.cipher()
                print(f"TLS: established ({proto}, {cipher[0]})")
    except Exception as e:
        print(f"TLS connect failed: {e}")

def log_response(resp: requests.Response, label: str, elapsed_ms: float | None = None):
    print(f"\n=== {label} RESPONSE ===")
    print(f"Status: {resp.status_code} {resp.reason}")
    print(f"Final URL: {redact_url(resp.url)}")
    if resp.history:
        print("Redirect chain:")
        for i, h in enumerate(resp.history, 1):
            print(f"  {i}. {h.status_code} -> {redact_url(h.url)}")
    if elapsed_ms is not None:
        print(f"Elapsed: {elapsed_ms:.1f} ms")
    print("\n-- Response headers --")
    for k, v in resp.headers.items():
        print(f"{k}: {v}")

    # Try JSON first
    body_snippet = None
    try:
        data = resp.json()
        print("\n-- JSON top-level keys --")
        print(list(data.keys()))
        # Helpful subsets
        if "error" in data:
            print("\n!! API reported error !!")
            print(json.dumps(data["error"], indent=2))
        if "search_parameters" in data:
            print("\nsearch_parameters:")
            print(json.dumps(data["search_parameters"], indent=2))
        if "search_metadata" in data:
            print("\nsearch_metadata:")
            print(json.dumps(data["search_metadata"], indent=2))
        if resp.status_code >= 400:
            print("\nFull JSON body on error:")
            print(json.dumps(data, indent=2)[:4000])
    except json.JSONDecodeError as e:
        text = resp.text or ""
        body_snippet = text[:2000]
        print(f"\n(JSON parse failed: {e})")
        print("\n-- Body (first 2000 chars) --")
        print(body_snippet)
    except Exception as e:
        print(f"\n(JSON handling error: {e})")

def show_prepared_request(prep: requests.PreparedRequest, label: str):
    print(f"\n=== {label} PREPARED REQUEST ===")
    print(f"{prep.method} {redact_url(prep.url)}")
    print("-- Request headers --")
    for k, v in prep.headers.items():
        print(f"{k}: {v}")

def try_head(url: str):
    print(f"\n=== HEAD {redact_url(url)} ===")
    try:
        resp = requests.head(url, timeout=TIMEOUT, allow_redirects=True)
        log_response(resp, "HEAD")
    except requests.exceptions.RequestException as e:
        print(f"HEAD failed: {e}")

def attempt_get_direct(url: str):
    print(f"\n=== Attempt #1: direct GET to provided URL ===")
    print(f"URL: {redact_url(url)}")
    s = requests.Session()
    req = requests.Request("GET", url)
    prep = s.prepare_request(req)
    show_prepared_request(prep, "Attempt #1")
    start = time.perf_counter()
    try:
        resp = s.send(prep, timeout=TIMEOUT, allow_redirects=True)
        elapsed = (time.perf_counter() - start) * 1000
        log_response(resp, "Attempt #1", elapsed_ms=elapsed)
        return resp
    except requests.exceptions.HTTPError as e:
        body = e.response.text if getattr(e, "response", None) is not None else ""
        print(f"HTTPError: {e}\n{body[:2000]}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")

def normalize_and_attempt_with_params(url: str):
    print(f"\n=== Attempt #2: normalized params via requests (fixing types/casing) ===")
    qs = parse_query(url)
    # Override API key from env if available (safer and avoids leaking if logs slip)
    if ENV_API_KEY:
        qs["api_key"] = ENV_API_KEY

    # Normalize booleans and ints to what SerpApi expects
    def as_bool_str(v):
        return "true" if str(v).lower() in ("1", "true", "yes") else "false"
    if "deep_search" in qs:
        qs["deep_search"] = as_bool_str(qs["deep_search"])

    for int_key in ("type", "travel_class", "adults", "children", "infants_in_seat", "infants"):
        if int_key in qs:
            try:
                qs[int_key] = int(qs[int_key])
            except Exception:
                pass

    # Prepare and send
    s = requests.Session()
    req = requests.Request("GET", BASE_URL, params=qs)
    prep = s.prepare_request(req)
    show_prepared_request(prep, "Attempt #2")
    start = time.perf_counter()
    try:
        resp = s.send(prep, timeout=TIMEOUT, allow_redirects=True)
        elapsed = (time.perf_counter() - start) * 1000
        log_response(resp, "Attempt #2", elapsed_ms=elapsed)
        return resp
    except requests.exceptions.HTTPError as e:
        body = e.response.text if getattr(e, "response", None) is not None else ""
        print(f"HTTPError: {e}\n{body[:2000]}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")

# ---- Run the diagnostics ----
print_env_and_connectivity()
try_head(RAW_URL)
resp1 = attempt_get_direct(RAW_URL)
resp2 = normalize_and_attempt_with_params(RAW_URL)

print("\n=== Summary ===")
print(f"Attempt #1 status: {getattr(resp1, 'status_code', 'N/A')}, URL: {redact_url(getattr(resp1, 'url', 'N/A'))}")
print(f"Attempt #2 status: {getattr(resp2, 'status_code', 'N/A')}, URL: {redact_url(getattr(resp2, 'url', 'N/A'))}")

print("\nTips:")
print("- If you see an 'error' field in JSON, it often includes the exact reason (key invalid, quota, unsupported param, etc.).")
print("- deep_search should be 'true'/'false'; Attempt #2 normalizes this.")
print("- If HTTP 401/403/429, check API key validity/quota and account status.")
print("- If JSON parse fails, inspect the raw body snippet above for HTML error pages or proxy errors.")

=== Environment ===
OS: Windows 10 (10.0.26100)
Python: 3.11.7
requests: 2.32.3
certifi: c:\Users\armoshar\AppData\Local\anaconda3\envs\gen-ai\Lib\site-packages\certifi\cacert.pem
SERPAPI_API_KEY in env: YES

=== DNS/TLS connectivity ===
serpapi.com resolves to: 162.159.142.21, 172.66.2.17


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): serpapi.com:443


TLS: established (TLSv1.3, TLS_AES_256_GCM_SHA384)

=== HEAD https://serpapi.com/search?engine=google_hotels&api_key=%2A%2A%2AREDACTED%2A%2A%2A&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&currency=USD&hl=en&gl=us ===


DEBUG:urllib3.connectionpool:https://serpapi.com:443 "HEAD /search?engine=google_hotels&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&currency=USD&hl=en&gl=us HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): serpapi.com:443
DEBUG:urllib3.connectionpool:https://serpapi.com:443 "GET /search?engine=google_hotels&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&currency=USD&hl=en&gl=us HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): serpapi.com:443
DEBUG:urllib3.connectionpool:https://serpapi.com:443 "GET /search?engine=google_hotels&api_key=d38b412680186df53a5ae8e0d24de852de5a26e4d6dab50cc93e55fa9c01d3d9&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&c


=== HEAD RESPONSE ===
Status: 200 OK
Final URL: https://serpapi.com/search?engine=google_hotels&api_key=%2A%2A%2AREDACTED%2A%2A%2A&q=Barcelona&check_in_date=2025-11-18&check_out_date=2025-11-21&adults=2&children=0&rooms=1&currency=USD&hl=en&gl=us

-- Response headers --
Date: Wed, 03 Sep 2025 00:37:22 GMT
Content-Type: application/json; charset=utf-8
Connection: keep-alive
Content-Encoding: br
x-frame-options: SAMEORIGIN
x-xss-protection: 1; mode=block
x-content-type-options: nosniff
x-download-options: noopen
x-permitted-cross-domain-policies: none
referrer-policy: strict-origin-when-cross-origin
x-robots-tag: noindex, nofollow
serpapi-search-id: 68b78dc0b2c613c11c5f3c8d
cache-control: max-age=3600, public
etag: W/"4e30a5c6db3de596638ed10a9ac37f02"
x-request-id: c93c3dac-c75f-44d2-9aa6-8a7b5c9e458a
x-runtime: 2.315837
CF-Cache-Status: MISS
Vary: Accept-Encoding
Server: cloudflare
CF-RAY: 97912d92395cf07a-DFW
alt-svc: h3=":443"; ma=86400

(JSON parse failed: Expecting value: line 1 co